In [2]:
import os
import cv2
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from src.configuration.config import datadict, TrainingDir
from src.Dataset.dataset import CustomDatasetHW, CustomDatasetHWD, CustomDataset, CustomDatasetHW_new, CustomDatasetHW_validation,CustomDatasetHW_3D
from src.utils.losses import BCEDiceLoss, DiceLoss, GeneralizedDiceLoss, WeightedCrossEntropyLoss, WeightedSmoothL1Loss
from src.configuration.config import IMAGE_HEIGHT, IMAGE_WIDTH,batch_size, num_workers, pin_memory, LEARNING_RATE
from src.utils.utils import custom_collate, custom_collate_Variable_HW
from src.Models.D_UNet import UNet3D,ResidualUNetSE3D

In [1]:
import sys 
sys.path.append(r'C:\Users\Rishabh\Documents\3D_Unet_Bleed')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DunetModel = UNet3D(in_channels=1, out_channels=1).to(device)
DunetModel = ResidualUNetSE3D(in_channels=1, out_channels=1).to(device)
# lossfun = GeneralizedDiceLoss()
lossfun = GeneralizedDiceLoss()

In [ ]:
TrainingDir = r'C:\Users\Rishabh\Documents\pytorch-3dunet\TrainingData'
ImagesDir = os.path.join(TrainingDir, 'Images')
MasksDir = os.path.join(TrainingDir, 'Masks')
dataset = CustomDatasetHW_3D(ImagesDir, MasksDir)

train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=True,
    collate_fn=custom_collate_Variable_HW,
)

In [ ]:
inputs, targets = next(iter(train_loader))

In [4]:
LEARNING_RATE = 0.0001

In [5]:
import torch.optim as optim

optimizer = optim.Adam(DunetModel.parameters(), lr=LEARNING_RATE)

# Set up Automatic Mixed Precision (AMP)
scaler = torch.cuda.amp.GradScaler()


C:\Users\Rishabh\AppData\Local\Temp\ipykernel_19400\2945504971.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
inputs = inputs[0,:,:,:,:]
targets = targets[0,:,:,:,:]

In [ ]:
inputs = inputs.unsqueeze(0)
targets = targets.unsqueeze(0)

In [ ]:
1e-2

In [ ]:
inputs = inputs[0,:,-8:,-16:,-16:]
targets = targets[0,:,-8:,-16:,-16:]

In [10]:
import torch

# Define shape
shape = (1, 1, 16, 64, 64)

# Tensor with random values
inputs = torch.rand(shape)

# Probability of 1 occurring is very low (e.g., 0.05)
probability_of_one = 0.0005  
targets = torch.bernoulli(torch.full(shape, probability_of_one)).to(torch.int)

print(torch.unique(targets, return_counts=True))


(tensor([0, 1], dtype=torch.int32), tensor([65508,    28]))


In [ ]:
DunetModel.train()
for i in range(10000):
    inputs = inputs.to(device)
    targets = targets.to(device)
    
    optimizer.zero_grad()
    
    # Forward pass with mixed precisio/n
    with torch.cuda.amp.autocast():
        outputs = DunetModel(inputs)
        loss = lossfun(outputs, targets)
    
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    if i%100 == 0:
        print(loss.item())

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_19400\2645584842.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


0.9991472363471985


In [ ]:
inputs.shape

In [ ]:
targets.shape

In [ ]:
inputs = np.array(inputs.cpu())


In [ ]:
targets = np.array(targets.cpu())

In [ ]:
np.unique(inputs)

In [ ]:
np.unique(targets)